In [44]:
PROMPT = """You are an ai trainer in marketplace company. You need to create a JSON from a plain text of a product offer provided by user. This JSON must contain next fields:
"place" - where buyer can recieve product (or delivery info)
"products" - list of dictionaries of valid products. 
Each product dict must contain ONLY following information: 
"name" - short title for a customer, 
"price" - cost of one unit of a product (one number), 
"currency" - in what currency is the price indicated (default is 'рубли'),
"size" - size or weight of a product (one string),
"count" - number of product units offered.
Represent different variations on each product as DIFFERENT products.
Do NOT add description and any other fields. Drop all fields with null value.
If NO information about the price provided, the product has to be excluded from the answer.
Answer ONLY with JSON string, no other text is needed.
"""

In [45]:
PROMPT2 = """You are an ai trainer in marketplace company. You need to create a JSON from a plain text of a product offer provided by user.
Let product UNIT be an unseparable set of objects being sold together. It does NOT matter how many UNITS of each type are available.
This JSON must contain next fields:
"place" - where buyer can recieve product or delivery info (if provided)
"products" - list of dictionaries of valid products. 
Each product dict must contain ONLY following information about ONE product UNIT: 
"name" (one string) - brief product UNIT's title for a customer. It must contain enough information for customer to understand what is being sold. Do NOT drop important general details, like product category, if provided; 
"count" (one string) - quantity of products being sold as a product UNIT.
"price" (one number) - cost of one product UNIT (for 'Count' quantity), 
"currency" (one string) - in what currency the 'Price' is indicated (default is 'RUB'),
Represent different variations on each product as DIFFERENT products.
Do NOT add description and any other fields. Drop all fields with null value.
If NO information about the price provided, the product has to be excluded from the answer.
Answer ONLY with JSON string, no other text is needed."""

In [144]:
PROMPT3 = """You are an AI trainer at a marketplace company. Your role is to convert product offer posts from plain text into structured JSON format. A "bundle" is defined as a set of items being sold together, which cannot be separated.

The JSON structure must include:

"place": The location where the buyer can pick up the product or delivery information, if available.
"bundles": A list of dictionaries, each representing a valid bundle.
Each bundle dictionary should contain ONLY the following information for ONE bundle:

"name" (string): A brief title of the bundle, including enough details for the customer to understand what is being sold. Ensure to include important general details, such as product category, if provided.
"quantity" (string): The amount of product in the bundle. For countable items, this is the number of items. For non-countable products (like liquids or materials), this should reflect the volume, length, or other relevant metrics.
"price" (number): The cost for the entire bundle, based on the 'quantity' specified.
"currency" (string): The currency for the 'price' (default is 'RUB').
Regarding product variations, such as size or color, these should NOT be treated as separate bundles. Variations should be considered additional descriptions and are not required in the JSON. The focus is on distinct products or bundles as defined by their essential characteristics, not their variations.

Exclude any fields with null values from the JSON. Do not add descriptions or any fields other than those specified. If no price information is provided, exclude the bundle from the JSON.

Your response should be a JSON string only, with no additional text required."""

In [189]:
PROMPT4 = """You are an AI trainer at a marketplace company, tasked with transforming product offer posts from plain text into structured JSON format. Define a "bundle" as a set of items sold together that cannot be separated.

The JSON output must include the following fields:

"place": The location for product pickup or delivery details, if provided.
"bundles": A list of dictionaries, with each dictionary representing a valid bundle.
Each dictionary in the "bundles" list should include ONLY the following information about ONE bundle:

"name" (string): A concise title for the bundle that includes enough detail for the customer to understand what is being sold. It's important to include general details such as product category, if available.
"quantity" (string): The amount of product in the bundle. For countable items, this represents the number of items. For non-countable products, such as liquids or materials, this should specify the volume, length, or other applicable metrics.
"price" (number): The total cost for the bundle, based on the specified 'quantity'.
"currency" (string): The currency of the 'price' (default is 'RUB').
Important clarifications:

Do not create separate bundles for product variations (e.g., size or color); these variations are not required in the JSON and should be regarded as additional descriptions.
If the post lacks clarity on what is being sold, specifically if understanding the offer depends on photos that are mentioned but not included in the text, do not add the product to the JSON. The description must enable a clear understanding of the product without reliance on external media.
Exclude any fields with null values. Do not include descriptions or fields other than those specified above. Exclude the bundle from the JSON if there is no price information.
Your response should be formatted as a JSON string, without any additional text."""

In [1]:
!pip install openai

In [396]:
import openai
import pickle as pkl
import pandas as pd
import json
import os
import numpy as np
from tqdm.notebook import tqdm

In [190]:
API_KEY = os.getenv('DIPLOMA_OPENAI_API_KEY')

In [191]:
openai_client = openai.OpenAI(api_key=API_KEY)

In [10]:
# with open('../cache/vk_filtered_1500.pkl', 'rb') as f:
#   df = pkl.load(f)
df = pd.read_csv('gpt_labeled_datas_small_v2.tsv', sep='\t')

In [185]:
df

,Text,Title,Price,Currency,Count,Label
0,Распродажа Костюм 2-ка Цена: 800р Ткань Флис...,Костюм 2-ка,800,RUB,1,1
1,Распродажа Набор коврик Для полок холодильни...,Набор коврик для полок холодильника,150,RUB,1,1
2,Распродажа Женские носки Размер 36-41 Цена у...,Женские носки Размер 36-41 Арт: 2А-25 корпус А,150,RUB,10 пар,1
3,Распродажа Цена 500р Цена за уп 6шт*450р Ткан...,Ткань велюр Арт 1Г-01/1А корпус А,500,RUB,1,1
4,Распродажа Цена 500р Цена за уп 6шт*450р Ткан...,Ткань велюр Арт 1Г-01/1А корпус А (упаковка),450,RUB,6,0
...,...,...,...,...,...,...
164,"продаю гель-лаки из личного пользования, во вс...",Гель-лак Mio Nails (цвет яркий коралловый),5,Lari,1,0
165,"продаю гель-лаки из личного пользования, во вс...","Гель-лак Venalisa (сплошные блёстки, серебро с...",5,Lari,1,0
166,"продаю гель-лаки из личного пользования, во вс...",Гель-лак Modelones (цвет серо-фиолетовый полуп...,5,Lari,1,0
167,"продаю гель-лаки из личного пользования, во вс...",Апельсиновые палочки,0,Lari,1,0


In [136]:
texts = []

In [154]:
texts2 = np.unique(df[df.Label == 0].Text.values)

In [135]:
print(df[(df.Label == 0)].iloc[-8].Text)

продам ящики.  два больших и один поменьше. 30 лари за 3 штуки. пишите в лс.


In [139]:
texts.append(df[df.Label == 0].iloc[-11].Text)

In [140]:
df_tmp = pd.DataFrame(texts, columns=['text'])
df_tmp

,text
0,продам ящики. два больших и один поменьше. 30...
1,"продаю гель-лаки из личного пользования, во вс..."
2,книги на английском blue moon 4€ the good fig...


In [155]:
df_tmp2 = pd.DataFrame(texts2, columns=['text'])
df_tmp2

,text
0,Доброй ночи. Продам 50р 120р за все Кружево 70...
1,Распродажа Цена 500р Цена за уп 6шт*450р Ткан...
2,вино сухое bianconero и jack rabbit 20 бутыл...
3,книги minecraft и коробочка mineworld. 15 евро...
4,книги на английском blue moon 4€ the good fig...
5,продам самокат xiaomi m365. все системные данн...
6,продам ящики. два больших и один поменьше. 30...
7,"продаю гель-лаки из личного пользования, во вс..."


In [141]:
from time import sleep

class OpenAIShooter:
  def __init__(self, client, prompt):
    self.client = client
    self.cache = {}
    self.model = "gpt-4-turbo-preview"
    self.prompt = prompt
    self.storage = {}

  def request(self, text):
    if text not in self.storage:
      retry = True
      while retry:
        global response
        response = self.client.chat.completions.create(
          model=self.model,
          messages=[
            {"role": "system", "content": self.prompt},
            {"role": "user", "content": text}
          ],
          **({'response_format':{"type": "json_object"}} if self.model == "gpt-3.5-turbo-1106" else {})
        )
        self.storage[text] = response
        try:
            retry = response.status_code != 200
        except:
            retry = False
        if retry:
          sleep(0.5)
    return self.storage[text]
  
  def shoot_all(self, df):
    answer = {}
    for id, text in df.text.items():
      resp = self.request(text).choices[0].message.content
      resp = re.search(r'\{.*\}', resp, re.DOTALL).group(0)
      try:
        answer[id] = json.loads(resp)
      except Exception as e:
        print(f'Fail: {e}')
        print(resp)
        # answer[id] = self.request(text)
    return answer

In [142]:
shooter = OpenAIShooter(openai_client, PROMPT2)

In [143]:
prompt1_res = shooter.shoot_all(df_tmp)
prompt1_res

{0: {'place': 'пишите в лс',
  'products': [{'name': 'ящики большие',
    'count': '2',
    'price': 20,
    'currency': 'лари'},
   {'name': 'ящик поменьше', 'count': '1', 'price': 10, 'currency': 'лари'}]},
 1: {'place': 'сабуртало, важа-пшавела метро',
  'products': [{'name': 'гель-лак moon - база молочная',
    'count': '1',
    'price': 30,
    'currency': 'лари'},
   {'name': 'гель-лак lac - цвет нежно розовый',
    'count': '1',
    'price': 30,
    'currency': 'лари'},
   {'name': 'гель-лак diva - густой топ с шиммером',
    'count': '1',
    'price': 30,
    'currency': 'лари'},
   {'name': 'гель-лак mio nails - цвет яркий коралловый',
    'count': '1',
    'price': 30,
    'currency': 'лари'},
   {'name': 'гель-лак venalisa - сплошные блёстки, серебро с красным',
    'count': '1',
    'price': 30,
    'currency': 'лари'},
   {'name': 'гель-лак modelones - цвет серо-фиолетовый полупрозрачный с блёстками',
    'count': '1',
    'price': 30,
    'currency': 'лари'},
   {'name': 

In [156]:
shooter3 = OpenAIShooter(openai_client, PROMPT3)
prompt3_res = shooter3.shoot_all(df_tmp2)
prompt3_res

{0: {'bundles': [{'name': 'Кружево',
    'quantity': '4м',
    'price': 120,
    'currency': 'RUB'},
   {'name': 'Журналы', 'quantity': '1шт', 'price': 40, 'currency': 'RUB'}]},
 1: {'place': None,
  'bundles': [{'name': 'Ткань велюр',
    'quantity': '6 шт',
    'price': 450,
    'currency': 'RUB'}]},
 2: {'place': 'лимассол',
  'bundles': [{'name': 'вино сухое bianconero и jack rabbit',
    'quantity': '20 бутылочек 250 мл',
    'price': 20,
    'currency': 'EUR'}]},
 3: {'place': 'никосия',
  'bundles': [{'name': 'книги minecraft и коробочка mineworld',
    'quantity': '1 set',
    'price': 15,
    'currency': 'EUR'}]},
 4: {'place': 'пафос, юниверсал',
  'bundles': [{'name': 'книги на английском blue moon',
    'quantity': '1',
    'price': 4,
    'currency': 'EUR'},
   {'name': 'книги на английском the good fight',
    'quantity': '1',
    'price': 5,
    'currency': 'EUR'},
   {'name': 'книги на английском the book thief',
    'quantity': '1',
    'price': 4,
    'currency': 'EUR

In [194]:
print(df_tmp2.iloc[0].text)

Доброй ночи. Продам 50р 120р за все Кружево 70р метр всего 4м 20р Журналы 40р 1шт 50р 160р 180р 120р


In [193]:
shooter4 = OpenAIShooter(openai_client, PROMPT4)
prompt4_res = shooter4.shoot_all(df_tmp2)
prompt4_res

{0: {'bundles': [{'name': 'Кружево',
    'quantity': '4м',
    'price': 280,
    'currency': 'RUB'},
   {'name': 'Журналы', 'quantity': '1шт', 'price': 40, 'currency': 'RUB'}]},
 1: {'bundles': [{'name': 'Ткань велюр размеры 48-50-52-54-56',
    'quantity': '6 шт',
    'price': 450,
    'currency': 'RUB'}]},
 2: {'place': 'лимассол',
  'bundles': [{'name': 'вино сухое bianconero и jack rabbit',
    'quantity': '20 бутылочек 250 мл',
    'price': 20,
    'currency': 'EUR'}]},
 3: {'place': 'никосия',
  'bundles': [{'name': 'книги minecraft и коробочка mineworld',
    'quantity': '1 комплект',
    'price': 15,
    'currency': 'EUR'}]},
 4: {'place': 'пафос, юниверсал',
  'bundles': [{'name': 'Book: Blue Moon',
    'quantity': '1',
    'price': 4,
    'currency': 'EUR'},
   {'name': 'Book: The Good Fight',
    'quantity': '1',
    'price': 5,
    'currency': 'EUR'},
   {'name': 'Book: The Book Thief',
    'quantity': '1',
    'price': 4,
    'currency': 'EUR'},
   {'name': 'Book: The Pill

In [223]:
df_texts = pd.DataFrame(np.unique(df.Text.values), columns=['text'])
shooter3 = OpenAIShooter(openai_client, PROMPT3)
prompt3_res_full = shooter3.shoot_all(df_texts)
prompt3_res_full

Fail: Expecting property name enclosed in double quotes: line 2 column 3 (char 4)
{
  \"place\": \"#лиговский _проспект\",
  \"bundles\": [
    {
      \"name\": \"Подарочный набор для второй половинки на 14 февраля, 23 февраля, 8 марта: баночка с записками '96 причин почему я тебя люблю', жевательные конфеты 'love is', открытка\",
      \"quantity\": \"1 набор\",
      \"price\": 500,
      \"currency\": \"RUB\"
    }
  ]
}


{0: {'place': 'делиси или важа пшавела',
  'bundles': [{'name': 'духи унисекс, оригинал',
    'quantity': 'больше 1/3 от 50мл',
    'price': 40,
    'currency': 'лари'},
   {'name': 'мерч футболка, унисекс, новая, размер l',
    'quantity': '1',
    'price': 40,
    'currency': 'лари'}]},
 1: {'place': None,
  'bundles': [{'name': 'Жидкости для вейпа (арбуз и ментол, смородина и яблоко)',
    'quantity': '2',
    'price': 500,
    'currency': 'RUB'}]},
 2: {'place': None,
  'bundles': [{'name': 'ipad air 64gb',
    'quantity': '1',
    'price': 350,
    'currency': 'RUB'}]},
 3: {'place': 'limassol',
  'bundles': [{'name': 'Full spearfishing equipment',
    'quantity': '1 set',
    'price': 80,
    'currency': 'EUR'}]},
 4: {'place': 'limassol',
  'bundles': [{'name': 'Mercedes Coupe 1998 SLK 230, Benzin',
    'quantity': '1',
    'price': 3000,
    'currency': 'EUR'}]},
 5: {'place': None,
  'bundles': [{'name': 'Xiaomi Mi Gaming Laptop Gen 2 15.6" (2019) with Intel Core i5-8300H 2.3 

In [233]:
import pickle as pkl

with open('prompt3_res_full.pkl', 'wb') as f:
  pkl.dump(prompt3_res_full, f)

In [279]:
df.iloc[40:50]

,Text,Title,Price,Currency,Count,Label
40,"Распродажа⚡ Платье Цена: 400руб. Размеры: 42,4...",Платье с пайетками размер 48,400,RUB,1,1
41,"Продам. Территориально Пушкин (Спб), отправлю ...",набор высечек фото1-6,100,RUB,1,1
42,"Продам. Территориально Пушкин (Спб), отправлю ...",набор высечек фото7,120,RUB,1,1
43,"Продам. Территориально Пушкин (Спб), отправлю ...",набор высечек фото8,140,RUB,1,1
44,"Продам. Территориально Пушкин (Спб), отправлю ...",набор высечек фото9-10,90,RUB,1,1
45,описание: продам книги харуки мураками. 1q84 ...,Книга '1q84' книга 1 Харуки Мураками,150,RUB,1,1
46,описание: продам книги харуки мураками. 1q84 ...,Книга '1q84' книга 3 Харуки Мураками,200,RUB,1,1
47,описание: продам книги харуки мураками. 1q84 ...,Книга 'Норвежский лес' Харуки Мураками,250,RUB,1,1
48,описание: продам книги харуки мураками. 1q84 ...,Книга 'Хроники заводной птицы. Птицелов' Харук...,100,RUB,1,1
49,описание: продам книги харуки мураками. 1q84 ...,Книга 'Хроники заводной птицы. Вещая птица' Ха...,50,RUB,1,1


In [246]:
res = prompt3_res_full
new_rows = []
for i in range(len(df_texts)):
  bundles = res[i]['bundles']
  for bundle in bundles:
    new_row = {
      'Text': df_texts.iloc[i].text,
      'Title': bundle['name'],
      'Quantity': bundle['quantity'],
      'Price': bundle['price'],
      'Currency': bundle['currency']
    }
    new_rows.append(new_row)

In [247]:
new_df = pd.DataFrame(new_rows)

In [248]:
new_df

,Text,Title,Quantity,Price,Currency
0,"1. духи унисекс, оригинал, остаток больше 1/3 ...","духи унисекс, оригинал",больше 1/3 от 50мл,40,лари
1,"1. духи унисекс, оригинал, остаток больше 1/3 ...","мерч футболка, унисекс, новая, размер l",1,40,лари
2,"2 жидкости для вейпа, заливала 1 раз, почти но...","Жидкости для вейпа (арбуз и ментол, смородина ...",2,500,RUB
3,ipad air 64gb. отличное состояние. 350л. пиш...,ipad air 64gb,1,350,RUB
4,"limassol, prodam polnuju ekipirovku dlia podvo...",Full spearfishing equipment,1 set,80,EUR
...,...,...,...,...,...
116,продаю 2 новые куртки (надевала 1-2 раза) 1. д...,Голубая дутая куртка Ostin на осень-зиму,1,75,GEL
117,"продаю гель-лаки из личного пользования, во вс...",Гель-лаки из личного пользования с апельсиновы...,6 шт + апельсиновые палочки,30,RUB
118,рожковая кофемашина ecp 33.21 280 лари,рожковая кофемашина ecp 33.21,1,280,RUB
119,"сумка от mango man для телефона, кошелька и пр...","Сумка Mango Man для телефона, кошелька и мелочей",1,50,GEL


In [249]:
labels = {}

In [401]:
def print_row(row):
  print(row.Text)
  print(f"""
    Title: {row.Title},
    Quantity: {row.Count},
    Price: {row.Price},
    Currency: {row.Currency},
  """)

In [399]:
new_df = pd.read_csv('gpt_labeled_datas_small_v3.tsv', sep='\t')
new_df

,Text,Title,Price,Currency,Count,Label
0,Распродажа Костюм 2-ка Цена: 800р Ткань Флис...,Распродажа Костюм 2-ка,800,RUB,1,1
1,Распродажа Набор коврик Для полок холодильни...,Набор коврик для полок холодильника,150,RUB,1,1
2,Распродажа Женские носки Размер 36-41 Цена у...,Женские носки размер 36-41,150,RUB,10 пар,1
3,Распродажа Цена 500р Цена за уп 6шт*450р Ткан...,"Ткань велюр, Арт 1Г-01/1А корпус А",450,RUB,6,1
4,Распродажа Платок Цена: 100р за упаковку 10ш...,Платок,100,RUB,10,1
...,...,...,...,...,...,...
113,Распродажа Трусы Утяжка Цена: 600р Цена упк: ...,Трусы Утяжка,3300,RUB,6,1
114,продам арт-бокс! хочешь открыть для себя новый...,"Арт-бокс для рисования картины ""лебеди в золоте""",1850,RUB,1,1
115,"limassol, prodam polnuju ekipirovku dlia podvo...",Полная экипировка для подводной рыбалки (подво...,80,EUR,1,1
116,"продаю гель-лаки из личного пользования, во вс...",гель-лаки с апельсиновыми палочками,30,Lari,6,1


In [404]:
labels = {}

In [402]:
for i in range(len(new_df)):
  print(f'ID: {i}')
  print_row(new_df.iloc[i])

ID: 0
Распродажа Костюм 2-ка  Цена: 800р  Ткань Флис  Размеры (46-48) (50-52) (54-56) Без выбора цвета Арт: 27-11 https://vk.com/fediy1995

    Title: Распродажа Костюм 2-ка,
    Quantity: 1,
    Price: 800,
    Currency: RUB,
  
ID: 1
Распродажа  Набор коврик  Для полок холодильника  Цена: 150р  Арт: 6-42

    Title: Набор коврик для полок холодильника,
    Quantity: 1,
    Price: 150,
    Currency: RUB,
  
ID: 2
Распродажа  Женские носки  Размер 36-41 Цена уп 10пар: 150р Без выбора рисунка Арт: 2А-25 корпус А https://vk.com/said2001mahmudov

    Title: Женские носки размер 36-41,
    Quantity: 10 пар,
    Price: 150,
    Currency: RUB,
  
ID: 3
Распродажа Цена 500р  Цена за уп 6шт*450р Ткань велюр  Размер 48-50-52-54-56 Арт 1Г-01/1А корпус А https://vk.com/huonghai

    Title: Ткань велюр, Арт 1Г-01/1А корпус А,
    Quantity: 6,
    Price: 450,
    Currency: RUB,
  
ID: 4
Распродажа  Платок  Цена: 100р за упаковку 10шт  Арт: 1Б-51 корпус А

    Title: Платок,
    Quantity: 10,
    Pr

In [417]:
i = 94
n = 1
for ii in range(i, i + n):
  labels[ii] = 0

In [418]:
labels

{22: 0,
 23: 0,
 24: 0,
 25: 0,
 35: 0,
 36: 0,
 37: 0,
 38: 0,
 39: 0,
 47: 0,
 59: 0,
 60: 0,
 61: 0,
 62: 0,
 63: 0,
 64: 0,
 65: 0,
 94: 0}

In [385]:
labels[i] = 1
i += 1
print(i-1, labels[i-1])

120 1


In [383]:
labels[i] = 0
i += 1
i
print(i-1, labels[i-1])

118 0


In [436]:
labels2 = labels

In [454]:
labels3 = labels2

In [455]:
for key in labels3:
  if labels3[key] == -1:
    labels3[key] = 'BA'
  if labels3[key] == 0:
    labels3[key] = 'MF'

In [456]:
new_df['Label'] = [labels3[i] if i in labels else 1 for i in range(len(new_df))]

In [457]:
new_df.to_csv('gpt_prompt_labeled_v3.csv')

In [458]:
new_df[new_df.Label != 1]

,Text,Title,Price,Currency,Count,Label
22,"Продам. Территориально Пушкин (Спб), отправлю ...",набор высечек (фото 1-6),100,RUB,1,BA
23,"Продам. Территориально Пушкин (Спб), отправлю ...",набор высечек (фото 7),120,RUB,1,BA
24,"Продам. Территориально Пушкин (Спб), отправлю ...",набор высечек (фото 8),140,RUB,1,BA
25,"Продам. Территориально Пушкин (Спб), отправлю ...",набор высечек (фото 9-10),90,RUB,1,BA
35,описание: привезли в столицу 💣модели. все фо...,guess,1000,RUB,1,BA
36,описание: привезли в столицу 💣модели. все фо...,snoopy mj,2800,RUB,1,BA
37,описание: привезли в столицу 💣модели. все фо...,michael kors сумки и рюкзаки,1800,RUB,1,BA
38,описание: привезли в столицу 💣модели. все фо...,соасh,3200,RUB,1,BA
39,описание: привезли в столицу 💣модели. все фо...,pinko,3500,RUB,1,BA
47,описание: новое поло tommy hilfiger размер 50...,"Б/у Поло белое, регби поло, поло синее размер ...",1000,RUB,1,MF


In [395]:
new_df.iloc[5:35]

,Text,Title,Quantity,Price,Currency,Label
5,mercedes coupe 1998. slk 230. benzin. low mile...,"Mercedes Coupe 1998 SLK 230, Benzin",1,3000,EUR,1
6,"xiaomi mi gaming laptop gen 2 15.6"" (2019) int...","Xiaomi Mi Gaming Laptop Gen 2 15.6"" (2019) wit...",1 unit,750,GEL,1
7,Доброй ночи. Продам 50р 120р за все Кружево 70...,Кружево,4м,280,RUB,1
8,Доброй ночи. Продам 50р 120р за все Кружево 70...,Журналы,1шт,40,RUB,1
9,Продам штампы и ножи оригиналы 1. Altenew 500р...,Altenew stamps and dies,1,500,RUB,1
10,Продам штампы и ножи оригиналы 1. Altenew 500р...,Pinkfresh/Altenew stamps and dies,2,800,RUB,0
11,Продам штампы и ножи оригиналы 1. Altenew 500р...,SSS stamps and dies,1,900,RUB,1
12,Продам штампы и ножи оригиналы 1. Altenew 500р...,SSS stamps without dies,1,700,RUB,1
13,Продам штампы и ножи оригиналы 1. Altenew 500р...,Altenew stamps and dies bundle,1,800,RUB,1
14,Продам штампы и ножи оригиналы 1. Altenew 500р...,MFT stamps and dies,1,700,RUB,1
